## Imports

In [1]:
from datasets import Dataset
from keras.utils.np_utils import to_categorical
from collections import Counter

## Constantes

In [25]:
dir = "../../"
dir_data = "data/"
dir_rotulacao = "rotulacao/result/"

arquivo_result = "twitter_34.json" ## Verificar caso a caso

dir_dataset = "datasets/dataset_twitter/"## Verificar caso a caso
dataset_name = "dataset_twitter.json" ## Verificar caso a caso

## Lendo Resultados

In [26]:
resultado_doccano = Dataset.from_json(f"{dir}{dir_data}{dir_rotulacao}{arquivo_result}")

Using custom data configuration default-6aed1ed55b747ac0
Found cached dataset json (/home/thiago/.cache/huggingface/datasets/json/default-6aed1ed55b747ac0/0.0.0)


In [27]:
resultado_doccano

Dataset({
    features: ['id', 'text', 'index', 'label', 'Comments'],
    num_rows: 34
})

## Lendo Dataset

In [28]:
dataset = Dataset.from_json(f"{dir}{dir_data}{dir_dataset}{dataset_name}")

Using custom data configuration default-7f8cf618f9c91533


HF google storage unreachable. Downloading and preparing it from source


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /home/thiago/.cache/huggingface/datasets/json/default-7f8cf618f9c91533/0.0.0. Subsequent calls will reuse this data.


In [29]:
dataset

Dataset({
    features: ['id', 'conversation_id', 'text', 'created_at', 'data_coleta', 'repetido', 'index', 'labels', 'labels_int', 'tweet_link'],
    num_rows: 17766
})

## Prepara Dados Resultados

In [30]:
resultado_doccano.features

{'id': Value(dtype='int64', id=None),
 'text': Value(dtype='string', id=None),
 'index': Value(dtype='int64', id=None),
 'label': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'Comments': Sequence(feature=Value(dtype='null', id=None), length=-1, id=None)}

### Remove indefinido

In [31]:
lista = []
for dado in resultado_doccano:
    lista.extend(dado["label"])
print(Counter(lista))

Counter({'Neutro': 26, 'Positivo': 7, 'Negativo': 1})


In [32]:
resultado_doccano = resultado_doccano.filter(lambda dado: "Indefinido" not in dado["label"])

  0%|          | 0/1 [00:00<?, ?ba/s]

In [33]:
lista = []
for dado in resultado_doccano:
    lista.extend(dado["label"])
print(Counter(lista))

Counter({'Neutro': 26, 'Positivo': 7, 'Negativo': 1})


### Arrumar Colunas de Label

In [34]:
resultado_doccano = resultado_doccano.add_column("labels_int", [None] * len(resultado_doccano))

Flattening the indices:   0%|          | 0/1 [00:00<?, ?ba/s]

In [35]:
def arruma_label(data):
    valor = 0
    if "Neutro" in data["label"]:
        valor = 1
    elif "Positivo" in data["label"]:
        valor = 2
    data["label"] = to_categorical(valor, num_classes=3)
    data["labels_int"] = valor
    return data
resultado_doccano = resultado_doccano.map(arruma_label)

  0%|          | 0/34 [00:00<?, ?ex/s]

In [36]:
Counter(resultado_doccano["labels_int"])

Counter({1: 26, 2: 7, 0: 1})

In [37]:
resultado_doccano.features

{'id': Value(dtype='int64', id=None),
 'text': Value(dtype='string', id=None),
 'index': Value(dtype='int64', id=None),
 'label': Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None),
 'Comments': Sequence(feature=Value(dtype='null', id=None), length=-1, id=None),
 'labels_int': Value(dtype='int64', id=None)}

In [38]:
Counter(resultado_doccano["labels_int"])

Counter({1: 26, 2: 7, 0: 1})

## Prepara Dados Dataset

In [39]:
dataset.features

{'id': Value(dtype='int64', id=None),
 'conversation_id': Value(dtype='int64', id=None),
 'text': Value(dtype='string', id=None),
 'created_at': Value(dtype='string', id=None),
 'data_coleta': Value(dtype='string', id=None),
 'repetido': Value(dtype='bool', id=None),
 'index': Value(dtype='int64', id=None),
 'labels': Sequence(feature=Value(dtype='float64', id=None), length=-1, id=None),
 'labels_int': Value(dtype='float64', id=None),
 'tweet_link': Value(dtype='string', id=None)}

### Adiciona a coluna de "label"

In [40]:
if "labels" not in dataset.features:
    dataset = dataset.add_column("labels", [None] * len(dataset))
    dataset = dataset.add_column("labels_int", [None] * len(dataset))

In [41]:
dataset.features

{'id': Value(dtype='int64', id=None),
 'conversation_id': Value(dtype='int64', id=None),
 'text': Value(dtype='string', id=None),
 'created_at': Value(dtype='string', id=None),
 'data_coleta': Value(dtype='string', id=None),
 'repetido': Value(dtype='bool', id=None),
 'index': Value(dtype='int64', id=None),
 'labels': Sequence(feature=Value(dtype='float64', id=None), length=-1, id=None),
 'labels_int': Value(dtype='float64', id=None),
 'tweet_link': Value(dtype='string', id=None)}

## Merge - Dataset & Result Doccano

### Convertendo result Doccano para Dict

In [42]:
dict_resultado_doccano = dict({dado["index"] : [dado["label"], dado["labels_int"], dado["text"]] for dado in resultado_doccano})

### Merge

In [43]:
def merge(data):
    if data["index"] in dict_resultado_doccano.keys() and not data["repetido"]:
        data["labels"] = dict_resultado_doccano[data["index"]][0]
        data["labels_int"] = dict_resultado_doccano[data["index"]][1]
        # print(f"texto doccano: {dict_resultado_doccano[data['index']][2]}")
        # print(f"texto dataset: {data['text']}")
        # print("-----------------------------------")
    return data
dataset = dataset.map(merge)

  0%|          | 0/17766 [00:00<?, ?ex/s]

## Salvando Dataset

In [44]:
dataset.to_json(f"{dir}{dir_data}{dir_dataset}{dataset_name}")

Creating json from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

7319054

## Testes

In [ ]:
from collections import Counter
print(Counter(dataset["labels_int"]))

In [ ]:
if "repetido" in dataset.features:
    dataset_repetido = dataset.filter(lambda data: data["repetido"])
    print(Counter(dataset_repetido["labels_int"]))

### Remover

In [ ]:
texto_remover = ""

In [ ]:
def remover_labels_by_texto(data, texto_remover: str, coluna: str):
    if texto_remover in data[coluna] and data["labels"]:
        data["labels"] = None
        data["labels_int"] = None
        print(f"Removeu labels do texto: {data[coluna]}")
    return data

dataset = dataset.map(lambda data: remover_labels_by_texto(data, texto_remover, "text"))


In [ ]:
cont = 0
for dado in resultado_doccano:
    if not dado["label"]:
        cont+= 1
print(cont)